<a href="https://colab.research.google.com/github/flying-bear/kompluxternaya/blob/master/assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 7

Train a Transformer model for Machine Translation from Russian to English.  
Dataset: http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz   
Make all source and target text to lower case.  

Score: corpus-bleu `nltk.translate.bleu_score.corpus_bleu`  
Use last 1000 sentences for model evalutation (test dataset).  
Use your target sequence tokenization for BLEU score.  
Use max_len=50 for sequence prediction.  


Hint: You may consider much smaller model, than shown in the example.  

Baselines:  
[4 point] BLEU = 0.05  
[6 point] BLEU = 0.10  
[9 point] BLEU = 0.15  

[1 point] Share weights between target embeddings and output dense layer. Notice, they have the same shape.


Readings:
1. BLUE score how to https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
1. Transformer code and comments http://nlp.seas.harvard.edu/2018/04/03/attention.html

In [1]:
!wget http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz

--2020-02-16 16:46:02--  http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113157482 (108M) [application/x-gzip]
Saving to: ‘training-parallel-nc-v13.tgz.4’

training-parallel-n 100%[===================>] 107.92M  96.3MB/s    in 1.1s    

2020-02-16 16:46:03 (96.3 MB/s) - ‘training-parallel-nc-v13.tgz.4’ saved [113157482/113157482]



In [2]:
!gunzip -c training-parallel-nc-v13.tgz | tar xvf - 

training-parallel-nc-v13/
training-parallel-nc-v13/news-commentary-v13.ru-en.ru
training-parallel-nc-v13/news-commentary-v13.cs-en.en
training-parallel-nc-v13/news-commentary-v13.de-en.de
training-parallel-nc-v13/news-commentary-v13.ru-en.en
training-parallel-nc-v13/news-commentary-v13.zh-en.zh
training-parallel-nc-v13/news-commentary-v13.zh-en.en
training-parallel-nc-v13/news-commentary-v13.cs-en.cs
training-parallel-nc-v13/news-commentary-v13.de-en.en


In [3]:
!rm training-parallel-nc-v13/news-commentary-v13.cs-en.en
!rm training-parallel-nc-v13/news-commentary-v13.de-en.de
!rm training-parallel-nc-v13/news-commentary-v13.zh-en.zh
!rm training-parallel-nc-v13/news-commentary-v13.zh-en.en
!rm training-parallel-nc-v13/news-commentary-v13.cs-en.cs
!rm training-parallel-nc-v13/news-commentary-v13.de-en.en

!ls training-parallel-nc-v13

news-commentary-v13.ru-en.en	   news-commentary-v13-test.ru-en.ru
news-commentary-v13.ru-en.ru	   news-commentary-v13-train.ru-en.en
news-commentary-v13-test.ru-en.en  news-commentary-v13-train.ru-en.ru


In [4]:
!pip install sentencepiece

In [0]:
import copy 

import math
import matplotlib.pyplot as plt
import numpy as np

import sentencepiece as spm
import seaborn

import time

import torch
import torch.nn as nn
import torch.nn.functional as F


from nltk.translate.bleu_score import corpus_bleu as bleu

from torch.autograd import Variable

from torchtext import data
from torchtext.data import Field
from torchtext.datasets import TranslationDataset

from tqdm import tqdm_notebook
from scipy.ndimage.filters import gaussian_filter1d


seaborn.set_context(context="talk")
%matplotlib inline

In [0]:
BATCH_SIZE = 128
max_len=50
max_src_in_batch = 25000
max_tgt_in_batch = 25000

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else tt.device('cpu')

## TOKENIZE

ENGLISH

In [8]:
spm.SentencePieceTrainer.Train('--input=training-parallel-nc-v13/news-commentary-v13.ru-en.en --model_prefix=bpe_en --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

True

In [9]:
tok_en = spm.SentencePieceProcessor()
tok_en.load('bpe_en.model')

True

In [0]:
TGT = Field(
    fix_length=max_len,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_en.encode_as_pieces(x),
    batch_first=True,
)

RUSSIAN

In [11]:
spm.SentencePieceTrainer.Train('--input=training-parallel-nc-v13/news-commentary-v13.ru-en.ru --model_prefix=bpe_ru --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

True

In [12]:
tok_ru = spm.SentencePieceProcessor()
tok_ru.load('bpe_ru.model')

True

In [0]:
TEXT = Field(
    fix_length=max_len,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_ru.encode_as_pieces(x),
    batch_first=True,
)

## DATASET

NB! every sentence is a new line

separate last 1000 sentences for test

In [0]:
with open('training-parallel-nc-v13/news-commentary-v13.ru-en.ru', 'r', encoding='utf-8') as f:
  source_text = f.readlines()

with open('training-parallel-nc-v13/news-commentary-v13-test.ru-en.ru', 'w', encoding='utf-8') as f:
  f.write(''.join(source_text[:1000]))

with open('training-parallel-nc-v13/news-commentary-v13-train.ru-en.ru', 'w', encoding='utf-8') as f:
  f.write(''.join(source_text[1000:]))

In [15]:
with open('training-parallel-nc-v13/news-commentary-v13-train.ru-en.ru', 'r', encoding='utf-8') as f:
  len_source = len(f.readlines())
print(len_source)

234159


In [0]:
with open('training-parallel-nc-v13/news-commentary-v13.ru-en.en', 'r', encoding='utf-8') as f:
  target_text = f.readlines()

with open('training-parallel-nc-v13/news-commentary-v13-test.ru-en.en', 'w', encoding='utf-8') as f:
  f.write(''.join(target_text[:1000]))

with open('training-parallel-nc-v13/news-commentary-v13-train.ru-en.en', 'w', encoding='utf-8') as f:
  f.write(''.join(target_text[1000:]))

In [17]:
with open('training-parallel-nc-v13/news-commentary-v13-train.ru-en.en', 'r', encoding='utf-8') as f:
  len_target = len(f.readlines())
print(len_target)

234159


In [0]:
dataset = TranslationDataset('training-parallel-nc-v13/news-commentary-v13-train', exts=('.ru-en.ru', '.ru-en.en'), fields=(TEXT, TGT))

In [0]:
trn, vld = dataset.split(split_ratio=0.8, stratified=False, random_state=None)

In [0]:
tst = TranslationDataset('training-parallel-nc-v13/news-commentary-v13-test', exts=('.ru-en.ru', '.ru-en.en'), fields=(TEXT, TGT))

In [0]:
TEXT.build_vocab(trn.src, min_freq=5)

In [0]:
TGT.build_vocab(trn.trg, min_freq=5)

In [0]:
pad_idx = TEXT.vocab.stoi["<pad>"]

In [0]:
vocab_tgt_size = len(TGT.vocab)
vocab_src_size = len(TEXT.vocab)

## MODEL ARCHITECTURE

In [0]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

### ENCODER

In [0]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [0]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [0]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [0]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [0]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [0]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### DECODER

In [0]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [0]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [0]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

### ATTENTION

In [0]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [0]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

### POSITION WISE

In [0]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [0]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [0]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

### ALL TOGETHER NOW

In [0]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

## BATCHES

In [0]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

## ITERATOR

In [0]:
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

def rebatch(pad_idx, batch):
    "Fix order in torchtext to match ours"
    src, trg = batch.src.transpose(0, 1), batch.trg.transpose(0, 1)
    return Batch(src, trg, pad_idx)

In [0]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

In [0]:
trn_itr = MyIterator(trn, batch_size=BATCH_SIZE, device=device,
                     repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                     batch_size_fn=None, train=True)
vld_itr = MyIterator(vld, batch_size=BATCH_SIZE, device=device,
                     repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                     batch_size_fn=None, train=False)
tst_itr = MyIterator(tst, batch_size=BATCH_SIZE, device=device,
                     repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                     batch_size_fn=None, train=False)

In [45]:
print(len(trn))
print(len(list(iter(trn_itr))))

187327
1464


In [46]:
print(len(vld))
print(len(list(iter(vld_itr))))

46832
366


In [47]:
example = next(iter(trn_itr))
example


[torchtext.data.batch.Batch of size 128]
	[.src]:[torch.cuda.LongTensor of size 128x50 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 128x50 (GPU 0)]

In [48]:
example.src

tensor([[   2,  156,   13,  ...,    1,    1,    1],
        [   2, 2733, 8947,  ...,    1,    1,    1],
        [   2,    5,   25,  ...,    1,    1,    1],
        ...,
        [   2,    5,   19,  ...,    1,    1,    1],
        [   2,    5,   19,  ...,    1,    1,    1],
        [   2, 2545,  128,  ...,    1,    1,    1]], device='cuda:0')

### TEXT HANDLER

In [0]:
def to_text(seq, field=TEXT):
  """gets a list of tokens given a list of items using a vocabulary of a given field
  
  :param seq: list of int, token ids
  :param field: trained torchtext.data.Field, optional, default TEXT
  :return: list of str, tokens
  """
  return [field.vocab.itos[v] for v in seq]

In [0]:
def join_tok(tok_list, eos_token='</s>', sos_token='</s>', pad_token='<pad>'): 
  """
  removes paading and joins subword tokens

  :param tok_list: list of str, subword tokens with "_" indicating a word beginning (space)
  :param eos_token: str, end of sequence token, optional, default </s> 
  :param sos_token: str, start of sequence token, optional, default <s> 
  :param pad_token: str, pad token, optional, default <pad> 
  :return: str, joined sentence
  """
  while tok_list[-1] == pad_token:
    tok_list = tok_list[:-1]
  if tok_list[-1] == eos_token:
    return ''.join(tok_list[:-1]).replace('▁', ' ').strip() + ' ' + eos_token
  else:
    return ''.join(tok_list).replace('▁', ' ').strip()

In [51]:
join_tok(to_text(example.src[0], TEXT))

'<s> кто-то может предложить ликвидацию налоговых льгот, чтобы сделать снижение ставки нейтральным для доходов бюджета. </s>'

In [52]:
join_tok(to_text(example.trg[0], TGT))

'<s> one can imagine closing loopholes to render rate cuts revenue neutral. but one person’s loophole is another’s entitlement. </s>'

## LOSS

In [0]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm, training=True):
        x = self.generator(x)
        loss = self.criterion(x.reshape(-1, x.size(-1)), y.reshape(-1)) / norm
        if training:
          loss.backward()
          if self.opt is not None:
              self.opt.step()
              self.opt.zero_grad()
        return loss.detach().item() * norm

## TRAINING

In [0]:
def run_epoch(data_iter, model, loss_compute, curr_epoch, n_batches, training=True):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    counter = 0
    running_loss = 0
    if training:
      iterator = tqdm_notebook(data_iter, total=n_batches, desc=f'epoch {curr_epoch + 1}', leave=True)
    else:
      iterator = tqdm_notebook(data_iter, total=n_batches, desc=f'eval epoch {curr_epoch + 1}', leave=True)
    train_losses = []
    for i, batch in enumerate(iterator):
        if batch.ntokens != 0:
          out = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
          loss = loss_compute(out, batch.trg_y, batch.ntokens, training=training)
          total_loss += loss
          total_tokens += batch.ntokens
          tokens += batch.ntokens
          train_losses.append(loss.item())

          running_loss = total_loss / total_tokens
          iterator.set_postfix(loss=f'{running_loss:.5f}')

          # if i % 100 == 1:
          #     elapsed = time.time() - start
          #     print(f"Epoch Step: {i} Loss on this batch: {loss} Time Elapsed {elapsed}") ## Tokens per Sec: {tokens / elapsed}
          #     # start = time.time()
          #     # tokens = 0
        else:
          counter += 1
          if counter % 100 == 0:
            print('100 zero-size batches')
    return running_loss, train_losses

In [0]:
model = make_model(vocab_src_size, vocab_tgt_size, N=2)
model.cuda()
# criterion = LabelSmoothing(vocab_tgt_size, padding_idx=pad_idx, smoothing=0.1)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx, reduction='sum')
criterion.cuda()
model_opt = torch.optim.Adam(model.parameters(), lr=0.05)

In [57]:
n_batches = len(list(iter(trn_itr)))
n_batches

1464

In [58]:
n_batches_vld = len(list(iter(vld_itr)))
n_batches_vld

366

In [0]:
train_history = []
eval_history = []
per_epoch_train_history = []
per_epoch_eval_history = []
for epoch in range(5):
        model.train()
        train_total_loss, train_losses = run_epoch((rebatch(pad_idx, b) for b in trn_itr), model, SimpleLossCompute(model.generator, criterion, opt=model_opt), epoch, n_batches)
        train_history += train_losses
        per_epoch_train_history.append(train_total_loss)
        with torch.no_grad():
          model.eval()
          eval_total_loss, eval_losses = run_epoch((rebatch(pad_idx, b) for b in vld_itr), model, SimpleLossCompute(model.generator, criterion, opt=None), epoch, n_batches_vld, training=False)
          eval_history += eval_losses
          per_epoch_eval_history.append(eval_total_loss)

In [68]:
print(len(train_history))
print(len(eval_history))
print(len(per_epoch_train_history))
print(len(per_epoch_eval_history))

2928
732
2
2


In [0]:
train_history

In [0]:
plt.figure(figsize=(14, 10))
plt.plot(range(int(len(train_history)/10)), [el.item() for el in train_history[::10]], np.array(range(int(len(eval_history)/10)))*4, [el.item() for el in eval_history[::10]])
plt.legend(('train loss', 'valid loss by batch'),
           loc='center', prop={'size': 18})
plt.title('Training process', fontsize=20)
plt.xlabel('Iterations', fontsize=16)
plt.ylabel('Loss function', fontsize=16)
plt.show()

In [0]:
smooth = lambda y: gaussian_filter1d(y, sigma=100)

plt.figure(figsize=(14, 10))
plt.plot(range(len(train_history)), smooth([el.item() for el in train_history]), np.array(range(len(eval_history)))*4, smooth([el.item() for el in eval_history]))
plt.legend(('train loss', 'valid loss by batch'),
           loc='center', prop={'size': 18})
plt.title('Smoothed training process', fontsize=20)
plt.xlabel('Iterations', fontsize=16)
plt.ylabel('Loss function (smoothed)', fontsize=16)
plt.show()

In [0]:
torch.save(model.state_dict(), 'model.pt')

In [0]:
model_loaded = make_model(vocab_src_size, vocab_tgt_size, N=2)
model_loaded.load_state_dict(torch.load('model.pt'))
model_loaded.eval()

## DECODE

In [0]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

model.eval()
with torch.no_grad():
  src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]]) )
  src_mask = Variable(torch.ones(1, 1, 10) )
  print(greedy_decode(model, src, src_mask, max_len=10, start_symbol=1))

ADAPT FROM LAST HW

In [0]:
def beam_search_from_start(model, k=3, max_len=30, eos='<eos>'):
  """
  autoregressive language modeling beam search decoder

  :param model: trained torch model, predicts logits torch tensor (of shape (batch, vocab, seq)) form batch tensor (of shape (batch, seq))
  :param k: int, beam width, optional, default 3
  :param max_len: int, maximal number of words produced, optional, default 30
  :param eos: str, end of sentence token, optional, default '<eos>'
  :return: list of str (of len k), top predicted sequences
  """
  id_eos = token_to_id(eos)
  with tt.no_grad():
    pred = model(tt.tensor([[start]]))[:,:,-1][0]
    toks = tt.argsort(pred, descending=True)[:k].tolist()
    tok_probs = pred[toks].tolist()
    candidates = [([start, toks[i]], tok_probs[i]) for i in range(len(toks))]
    while max_len > max([len(seq) for seq, prob in candidates]):
      # expand each current candidate that doesn't have an EOS at the end
      new_candidates = []
      for seq, prob in candidates:
        if seq[-1] != id_eos:
          pred = model(tt.tensor([seq]))[:,:,-1][0]
          toks = tt.argsort(pred, descending=True)[:k].tolist()
          # probs = np.log(soft_with_temp(pred))
          tok_probs = pred[toks].tolist()
          new_candidates += [(seq[:]+[toks[i]], prob+tok_probs[i]) for i in range(len(toks))]
        else:
          new_candidates += [(seq, prob)]
      candidates = sorted(new_candidates, key=lambda tup:tup[1])[:k] # order all candidates by score and select k best
  return [join_tok(to_text(seq)) for seq, prob in candidates]